In [1]:
import numpy as np
import torch
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import re
import os
from itertools import permutations
import pandas as pd

from fmri_reconstruction_with_dmvae.deeprecon.datasets.load import load_data
from fmri_reconstruction_with_dmvae.deeprecon.datasets.align import get_train_data, get_test_data
from fmri_reconstruction_with_dmvae.deeprecon.datasets.dataset import get_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

### Data

In [2]:
data_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/data/deeprecon/"
subj_list = [1, 2, 3, 4, 5]  # [1, 2, 3, 4, 5]

num_samples = 2400

# set needed repetitions
rep = 1 if num_samples < 1200 else int(num_samples / 1200)

In [3]:
### Load
train_brain_data_dict, test_brain_data_dict, all_subj_num_voxels = load_data(data_path, subj_list)

In [4]:
### Align
train_data, train_mean_dict, train_norm_dict = get_train_data(train_brain_data_dict, subj_list, rep, is_normalized=True)
test_data = get_test_data(test_brain_data_dict, subj_list, train_mean_dict, train_norm_dict, is_normalized=True)

In [5]:
### Dataset
train_dataset, test_dataset = get_dataset(train_data, test_data)

In [6]:
### DataLoader
batch_size = 128
g = torch.Generator()

train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, generator=g)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True)

### Model & Train & Evaluation

#### Function

In [ ]:
def train(epoch):
    train_loss = 0
    total_samples = 0

    for data in tqdm(train_dl):
        optimizer.zero_grad()

        source_batch = data[f"subj{s_s}"].to(torch.float32).to(device)
        target_batch = data[f"subj{s_t}"].to(torch.float32).to(device)
        recon_batch = model(source_batch)
        
        loss = loss_fn(recon_batch, target_batch)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * recon_batch.size(0)
        total_samples += recon_batch.size(0)

    train_loss = train_loss / total_samples

    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

def test(epoch):
    model.eval()
    test_loss = 0
    total_samples = 0

    target_brain = []
    recon_brain = []
    label = []

    with torch.no_grad():
        for data in tqdm(test_dl):
            source_batch = data[f"subj{s_s}"].to(torch.float32).to(device)
            target_batch = data[f"subj{s_t}"].to(torch.float32).to(device)
            recon_batch = model(source_batch)

            loss = loss_fn(recon_batch, target_batch)

            test_loss += loss.item() * recon_batch.size(0)
            total_samples += recon_batch.size(0)

            if epoch == epochs:
                target_brain.append(target_batch.cpu())
                recon_brain.append(recon_batch.cpu())
                label.append(data["image_index"].cpu())
            
    test_loss = test_loss / total_samples

    print('Epoch: {} Test loss: {:.4f}'.format(epoch, test_loss))
    return test_loss, target_brain, recon_brain, label

In [ ]:
def calculate_pattern_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]
    label = label[sort_idx]
    unique_label = np.unique(label)

    pattern_corr = []
    for image_idx in unique_label:
        target_pattern = target_brain[(label == image_idx).flatten(), :]
        recon_pattern = recon_brain[(label == image_idx).flatten(), :]
        
        corrs = np.corrcoef(target_pattern, recon_pattern)[:rep, rep:]
        corr = np.mean(corrs) 

        pattern_corr.append(corr)

    return pattern_corr

def calculate_profile_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]

    profile_corr = []
    for voxel_idx in range(target_brain.shape[1]):
        target_profile = target_brain[:, voxel_idx].reshape(rep, -1, order="F")
        recon_profile = recon_brain[:, voxel_idx].reshape(rep, -1, order="F")
        
        corrs = np.corrcoef(target_profile, recon_profile)[:rep, rep:]
        corr = np.mean(corrs) 

        profile_corr.append(corr)

    return profile_corr

def save_result(result_data, output_dir, output_filename):
    """
    Save the results to a CSV file.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df = pd.DataFrame(result_data)
    df.to_csv(os.path.join(output_dir, output_filename), index=None)

#### Converter1

In [26]:
from fmri_reconstruction_with_dmvae.models.converter import Converter1

lr = 1e-4
weight_decay = 1e-1

epochs = 7

conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"

pattern_corr_result = []
profile_corr_result = []

for subj_target, subj_source in permutations(subj_list, 2):
    s_t = f"{int(subj_target):02d}"
    s_s = f"{int(subj_source):02d}"

    model = Converter1(all_subj_num_voxels[f"subj{s_s}"], all_subj_num_voxels[f"subj{s_t}"]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # L2
    loss_fn = torch.nn.MSELoss()

    g.manual_seed(42)
    for epoch in range(1, epochs + 1):
        train_loss = train(epoch)
        test_loss, target_brain, recon_brain, label = test(epoch)

    target_brain = torch.cat(target_brain, dim=0).cpu().numpy()
    recon_brain = torch.cat(recon_brain, dim=0).cpu().numpy()
    label = torch.cat(label, dim=0).cpu().numpy()

    pattern_corr = calculate_pattern_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(pattern_corr):
        pattern_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Image_idx': i+1}
        )

    profile_corr = calculate_profile_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(profile_corr):
        profile_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Voxel_idx': i}
        )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter1.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter1.csv")

100%|██████████| 18/18 [00:00<00:00, 30.61it/s]


Epoch: 1 Train loss: 1.2342


100%|██████████| 10/10 [00:00<00:00, 56.13it/s]


Epoch: 1 Test loss: 1.2247


100%|██████████| 18/18 [00:00<00:00, 27.92it/s]


...

Epoch: 6 Test loss: 1.2806


100%|██████████| 18/18 [00:00<00:00, 31.99it/s]


Epoch: 7 Train loss: 0.9893


100%|██████████| 10/10 [00:00<00:00, 44.17it/s]


Epoch: 7 Test loss: 1.2800


#### Converter2

In [30]:
from fmri_reconstruction_with_dmvae.models.converter import Converter2

hidden_dim = 4096

lr = 1e-4
weight_decay = 1e-1

epochs = 7

conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"

pattern_corr_result = []
profile_corr_result = []

for subj_target, subj_source in permutations(subj_list, 2):
    s_t = f"{int(subj_target):02d}"
    s_s = f"{int(subj_source):02d}"

    model = Converter2(all_subj_num_voxels[f"subj{s_s}"], all_subj_num_voxels[f"subj{s_t}"], hidden_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay = weight_decay) # L2
    loss_fn = torch.nn.MSELoss()

    g.manual_seed(42)
    for epoch in range(1, epochs + 1):
        train_loss = train(epoch)
        test_loss, target_brain, recon_brain, label = test(epoch)

    target_brain = torch.cat(target_brain, dim=0).cpu().numpy()
    recon_brain = torch.cat(recon_brain, dim=0).cpu().numpy()
    label = torch.cat(label, dim=0).cpu().numpy()

    pattern_corr = calculate_pattern_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(pattern_corr):
        pattern_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Image_idx': i+1}
        )

    profile_corr = calculate_profile_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(profile_corr):
        profile_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Voxel_idx': i}
        )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter2.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter2.csv")

100%|██████████| 18/18 [00:00<00:00, 39.99it/s]


Epoch: 1 Train loss: 1.0336


100%|██████████| 10/10 [00:00<00:00, 54.31it/s]


Epoch: 1 Test loss: 1.0126


100%|██████████| 18/18 [00:00<00:00, 36.23it/s]


...

Epoch: 6 Test loss: 1.2795


100%|██████████| 18/18 [00:00<00:00, 41.61it/s]


Epoch: 7 Train loss: 0.9971


100%|██████████| 10/10 [00:00<00:00, 49.71it/s]


Epoch: 7 Test loss: 1.2795


#### Converter3

In [31]:
from fmri_reconstruction_with_dmvae.models.converter import Converter3

hidden_dim = 4096

lr = 1e-4
weight_decay = 1e-1

epochs = 7

conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"

pattern_corr_result = []
profile_corr_result = []

for subj_target, subj_source in permutations(subj_list, 2):
    s_t = f"{int(subj_target):02d}"
    s_s = f"{int(subj_source):02d}"

    model = Converter3(all_subj_num_voxels[f"subj{s_s}"], all_subj_num_voxels[f"subj{s_t}"], hidden_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = torch.nn.MSELoss()

    g.manual_seed(42)
    for epoch in range(1, epochs + 1):
        train_loss = train(epoch)
        test_loss, target_brain, recon_brain, label = test(epoch)

    target_brain = torch.cat(target_brain, dim=0).cpu().numpy()
    recon_brain = torch.cat(recon_brain, dim=0).cpu().numpy()
    label = torch.cat(label, dim=0).cpu().numpy()

    pattern_corr = calculate_pattern_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(pattern_corr):
        pattern_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Image_idx': i+1}
        )

    profile_corr = calculate_profile_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(profile_corr):
        profile_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Voxel_idx': i}
        )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter3.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter3.csv")

100%|██████████| 18/18 [00:00<00:00, 28.26it/s]


Epoch: 1 Train loss: 1.0045


100%|██████████| 10/10 [00:00<00:00, 55.52it/s]


Epoch: 1 Test loss: 0.9875


100%|██████████| 18/18 [00:00<00:00, 37.77it/s]


...

Epoch: 6 Test loss: 1.2795


100%|██████████| 18/18 [00:00<00:00, 39.69it/s]


Epoch: 7 Train loss: 0.9972


100%|██████████| 10/10 [00:00<00:00, 44.56it/s]


Epoch: 7 Test loss: 1.2795


#### Converter4

In [32]:
from fmri_reconstruction_with_dmvae.models.converter import Converter4

zp_dim = 256
zs_dim = 768
hidden_dim = 1024

lr = 1e-4
weight_decay = 1e-1

epochs = 7

conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"

pattern_corr_result = []
profile_corr_result = []

for subj_target, subj_source in permutations(subj_list, 2):
    s_t = f"{int(subj_target):02d}"
    s_s = f"{int(subj_source):02d}"

    model = Converter4(all_subj_num_voxels[f"subj{s_s}"], all_subj_num_voxels[f"subj{s_t}"], zp_dim, zs_dim, hidden_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = torch.nn.MSELoss()

    g.manual_seed(42)
    for epoch in range(1, epochs + 1):
        train_loss = train(epoch)
        test_loss, target_brain, recon_brain, label = test(epoch)

    target_brain = torch.cat(target_brain, dim=0).cpu().numpy()
    recon_brain = torch.cat(recon_brain, dim=0).cpu().numpy()
    label = torch.cat(label, dim=0).cpu().numpy()

    pattern_corr = calculate_pattern_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(pattern_corr):
        pattern_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Image_idx': i+1}
        )

    profile_corr = calculate_profile_correlation(target_brain, recon_brain, label)
    for i, corr in enumerate(profile_corr):
        profile_corr_result.append({
            'Subject_target': subj_target, 
            'Subject_source': subj_source,           
            'Correlation': corr, 
            'Voxel_idx': i}
        )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter4.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter4.csv")

100%|██████████| 18/18 [00:00<00:00, 34.51it/s]


Epoch: 1 Train loss: 1.0005


100%|██████████| 10/10 [00:00<00:00, 45.55it/s]


Epoch: 1 Test loss: 0.9850


100%|██████████| 18/18 [00:00<00:00, 28.53it/s]


...

Epoch: 6 Test loss: 1.2796


100%|██████████| 18/18 [00:00<00:00, 39.03it/s]


Epoch: 7 Train loss: 0.9973


100%|██████████| 10/10 [00:00<00:00, 48.19it/s]


Epoch: 7 Test loss: 1.2796


### Single

In [ ]:
from fmri_reconstruction_with_dmvae.models.converter import Converter

lr = 1e-4

subj_target = 1
subj_source = 2

s_t = f"{int(subj_target):02d}"
s_s = f"{int(subj_source):02d}"

zp_dim = 256
zs_dim = 768
hidden_dim = 1024
model = Converter(all_subj_num_voxels[f"subj{s_s}"], all_subj_num_voxels[f"subj{s_t}"], zp_dim, zs_dim, hidden_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()

In [ ]:
def train(epoch):
    train_loss = 0
    total_samples = 0

    for data in tqdm(train_dl):
        optimizer.zero_grad()

        source_batch = data[f"subj{s_s}"].to(torch.float32).to(device)
        target_batch = data[f"subj{s_t}"].to(torch.float32).to(device)
        recon_batch = model(source_batch)
        
        loss = loss_fn(recon_batch, target_batch)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * recon_batch.size(0)
        total_samples += recon_batch.size(0)

    train_loss = train_loss / total_samples

    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    total_samples = 0

    target_brain = []
    recon_brain = []
    label = []

    with torch.no_grad():
        for data in tqdm(test_dl):
            source_batch = data[f"subj{s_s}"].to(torch.float32).to(device)
            target_batch = data[f"subj{s_t}"].to(torch.float32).to(device)
            recon_batch = model(source_batch)

            loss = loss_fn(recon_batch, target_batch)

            test_loss += loss.item() * recon_batch.size(0)
            total_samples += recon_batch.size(0)

            if epoch == epochs:
                target_brain.append(target_batch.cpu())
                recon_brain.append(recon_batch.cpu())
                label.append(data["image_index"].cpu())
            
    test_loss = test_loss / total_samples

    print('Epoch: {} Test loss: {:.4f}'.format(epoch, test_loss))
    return test_loss, target_brain, recon_brain, label

In [70]:
epochs = 10
g.manual_seed(42)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss, target_brain, recon_brain, label = test(epoch)

target_brain = torch.cat(target_brain, dim=0).cpu().numpy()
recon_brain = torch.cat(recon_brain, dim=0).cpu().numpy()
label = torch.cat(label, dim=0).cpu().numpy()

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:00<00:00, 24.00it/s]


Epoch: 1 Train loss: 114.3821


100%|██████████| 10/10 [00:00<00:00, 29.89it/s]


Epoch: 1 Test loss: 223.3641


...

Epoch: 9 Test loss: 223.7538


100%|██████████| 18/18 [00:00<00:00, 32.30it/s]


Epoch: 10 Train loss: 92.0130


100%|██████████| 10/10 [00:00<00:00, 43.51it/s]

Epoch: 10 Test loss: 224.2601


In [71]:
def calculate_pattern_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]
    label = label[sort_idx]
    unique_label = np.unique(label)

    pattern_corr = []
    for image_idx in unique_label:
        target_pattern = target_brain[(label == image_idx).flatten(), :]
        recon_pattern = recon_brain[(label == image_idx).flatten(), :]
        
        corrs = np.corrcoef(target_pattern, recon_pattern)[:rep, rep:]
        corr = np.mean(corrs[np.triu_indices(rep, k=0)]) # including the diagonal

        pattern_corr.append(corr)

    return pattern_corr

In [72]:
def calculate_profile_correlation(target_brain, recon_brain, label, rep=24):
    sort_idx = np.argsort(label.flatten())
    target_brain = target_brain[sort_idx]
    recon_brain = recon_brain[sort_idx]

    profile_corr = []
    for voxel_idx in range(target_brain.shape[1]):
        target_profile = target_brain[:, voxel_idx].reshape(rep, -1, order="F")
        recon_profile = recon_brain[:, voxel_idx].reshape(rep, -1, order="F")
        
        corrs = np.corrcoef(target_profile, recon_profile)[:rep, rep:]
        corr = np.mean(corrs[np.triu_indices(rep, k=0)]) # including the diagonal

        profile_corr.append(corr)

    return profile_corr

In [73]:
def save_result(result_data, output_dir, output_filename):
    """
    Save the results to a CSV file.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df = pd.DataFrame(result_data)
    df.to_csv(os.path.join(output_dir, output_filename), index=None)

In [76]:
conversion_output_path = "/home/acg17270jl/projects/fmri-reconstruction-with-dmvae/results/deeprecon/conversion/"

pattern_corr_result = []
profile_corr_result = []

pattern_corr = calculate_pattern_correlation(target_brain, recon_brain, label)
for i, corr in enumerate(pattern_corr):
    pattern_corr_result.append({
        'Subject_target': subj_target, 
        'Subject_source': subj_source,           
        'Correlation': corr, 
        'Image_idx': i+1}
    )

profile_corr = calculate_profile_correlation(target_brain, recon_brain, label)
for i, corr in enumerate(profile_corr):
    profile_corr_result.append({
        'Subject_target': subj_target, 
        'Subject_source': subj_source,           
        'Correlation': corr, 
        'Voxel_idx': i}
    )

save_result(pattern_corr_result, conversion_output_path, "pattern_correlation_converter3.csv")
save_result(profile_corr_result, conversion_output_path, "profile_correlation_converter3.csv")